In [9]:
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

In [10]:
(pd.read_csv('https://raw.githubusercontent.com/dev-achintha/Sri_Lanka-Tourism_Forcasting_Model/prophet/dataset/2014-2024-monthly-tourist-arrivals-sl-csv.csv')).head()

,Year,Month,Arrivals
0,2014,January,146575
1,2014,February,141878
2,2014,March,133048
3,2014,April,112631
4,2014,May,90046


In [12]:
(pd.read_csv('https://raw.githubusercontent.com/dev-achintha/Sri_Lanka-Tourism_Forcasting_Model/prophet/dataset/2018-2024-Country-based-monthly-tourist-arrivals-sl-csv.csv')).head()

,YEAR,COUNTRY,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBER,DECEMBER,TOTAL UP TO END OF DECEMBER
0,2018,AFGHANISTAN,83.0,150.0,112.0,40.0,108.0,44.0,47.0,78.0,34.0,41,52,72,861.0
1,2019,AFGHANISTAN,49.0,47.0,59.0,38.0,2.0,12.0,47.0,54.0,50.0,22,55,38,473.0
2,2020,AFGHANISTAN,65.0,64.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,146.0
3,2021,AFGHANISTAN,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,4.0,0,1,5,15.0
4,2022,AFGHANISTAN,0.0,2.0,16.0,0.0,1.0,0.0,2.0,0.0,0.0,0,7,11,39.0


In [ ]:
# Load the first dataset (2014-2024 monthly tourist arrivals)
df1 = pd.read_csv('https://raw.githubusercontent.com/dev-achintha/Sri_Lanka-Tourism_Forcasting_Model/prophet/dataset/2014-2024-monthly-tourist-arrivals-sl-csv.csv')
df1 = df1.drop(columns=['PercentageChange'], errors='ignore')
df1['ds'] = pd.to_datetime(df1['Year'].astype(str) + '-' + df1['Month'], format='%Y-%B')
df1.rename(columns={'Arrivals': 'y'}, inplace=True)
df1 = df1[['ds', 'y']]

In [8]:
# Load the second dataset (2018-2024 country-based monthly tourist arrivals)
df2 = pd.read_csv('https://raw.githubusercontent.com/dev-achintha/Sri_Lanka-Tourism_Forcasting_Model/prophet/dataset/2018-2024-Country-based-monthly-tourist-arrivals-sl-csv.csv')

# Melt the second dataset to long format
df2_melted = df2.melt(id_vars=['YEAR', 'COUNTRY'], 
                      var_name='MONTH', 
                      value_name='ARRIVALS')

In [ ]:

df2_melted.to_csv('melted_tourist_arrivals.csv', index=False)

In [ ]:
# Convert month names to numbers
month_dict = {
    'JANUARY': 1, 'FEBRUARY': 2, 'MARCH': 3, 'APRIL': 4, 'MAY': 5, 'JUNE': 6,
    'JULY': 7, 'AUGUST': 8, 'SEPTEMBER': 9, 'OCTOBER': 10, 'NOVEMBER': 11, 'DECEMBER': 12
}

# Convert month names to numbers, handling non-month values
df2_melted['MONTH'] = df2_melted['MONTH'].apply(lambda x: month_dict.get(str(x).upper(), np.nan))

# Remove rows with invalid months
# df2_melted = df2_melted.dropna(subset=['MONTH'])

df2_melted.head()

In [ ]:
from prophet import Prophet

model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False,
                changepoint_prior_scale=0.1, n_changepoints=30)
model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
# changepoints = ['2019-04-21', '2020-03-01', '2022-03-22', '2023-02-25']
#     2019-04-21 Easter Bombings
#     2020-03-01 Covid19 Pandemic
#     2022-03-22 Dollar rate increses
#     2023-02-25 Dollar rate decreases
# model = Prophet(changepoints=changepoints)

model.fit(df)

In [ ]:
future_periods = 24

future = model.make_future_dataframe(periods=future_periods, freq='M')

forecast = model.predict(future)

forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
import matplotlib.pyplot as plt

fig = model.plot(forecast)
plt.title('Tourism Forecast')
plt.xlabel('Date')
plt.ylabel('Tourist Arrivals')
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(df['ds'], df['y'], label='Actual Arrivals', color='blue')

ax.plot(forecast['ds'], forecast['yhat'], label='Predicted Arrivals', color='red')

ax.set_title('Actual vs Predicted Tourist Arrivals')
ax.set_xlabel('Date')
ax.set_ylabel('Arrivals')
ax.legend()

plt.show()


In [ ]:
from sklearn.metrics import mean_absolute_error

actual = df['y']
predicted = forecast.loc[forecast['ds'].isin(df['ds']), 'yhat']

mae = mean_absolute_error(actual, predicted)
print(f"Mean Absolute Error: {mae}")


In [ ]:
!apt-get install -y pandoc texlive-xetex texlive-fonts-recommended texlive-plain-generic

In [ ]:
!jupyter nbconvert --to pdf /content/Sri_Lankan_TourismForecastModel_Prophet.ipynb